---
- TOÁN ỨNG DỤNG THỐNG KÊ
- Lớp: 22_3
- Họ và tên: Nguyễn Minh Thiện
- MSSV: 22120344
- Lab 04: Đồ án Diagonalizable matrix
---

### Bài 1)
### Giải thuật của thuật toán Diagonalizable matrix 
- **Bước 1:** Tìm đa thức đặc trưng $det(A − λI)$. Nếu $P_A(λ)$ có tổng các lũy thừa khác $n$ thì $A$ không chéo hóa được và kết thúc thuật toán, ngược lại chuyển sang **Bước 2**.
- **Bước 2:** Tìm tất cả các nghiệm $λ_i$ của phương trình đa thức đặc trưng. Với mỗi trị riêng $λ_i$ tìm cơ sở và số chiều cho không gian nghiệm phương trình $(A − λI_n) X = 0$. Nếu mỗi $λ_i$ số chiều không gian nghiệm nhỏ hơn lũy thừa của $λ_i$ trong đa thức đặc trưng thì $A$ không chéo hóa được và thuật toán kết thúc, ngược lại chuyển sang **Bước 3**.
- **Bước 3:** Với các vector trong cơ sở không gian nghiệm tìm được ở bước 2, ta đặt ma trận $P$ là ma trận có được bằng cách dựng các vector thành các cột. Khi đó ma trận $P$ làm chéo $A$ và $D = P^{−1}AP$ là ma trận đường chéo. $diag(λ_1, . . . , λ_r)$


In [36]:
import copy
import sympy as sp

# Hàm chuyển số thực hành phân số 
# Lưu ý: Đôi khi các số như sqrt(3) ở dạng số thực thì được chuyển sang dạng phân số có kết quả xấp xỉ 

def float_to_fraction(x):
    if x % 1 == 0:
        return str(int(x))
    saiSo = 1.0E-6
    num, denom = 1, 1
    decimal, sign = abs(x), -1 if x < 0 else 1
    while abs(decimal - num/denom) > saiSo:
        if decimal < num/denom:
            denom += 1
        else:
            num += 1
    return f"{sign * num}/{denom}"
    

# Hàm in ma trận
def printMatrix(A):
    if A is None:
        print('\n')
    else:
        for row in A:
            print([float_to_fraction(elem) for elem in row])
        print('\n')

# Hàm tạo ma trận đơn vị cấp n
def createIdentityMatrix(n):
    A=[]
    for i in range(0,n):
        temp=[]
        for j in range(0,n):
            if(i==j):
                temp.append(1)
            else:
                temp.append(0)            
        A.append(temp)
    return A


def inverse(A):
    # Kiểm tra nếu ma trận A input không hợp lệ
    for i in range(len(A)):
        if len(A) != len(A[i]):
            print("Ma trận input không vuông, không thể tìm ma trận nghịch đảo!")
            return None
    
    # Tạo ma trận đơn vị In
    B = createIdentityMatrix(len(A))
        
    flag = True
    j = 0
    
    # Duyệt qua từng cột của ma trận A
    for i in range(len(A[0])):
        k = j
        flag = False
        
        # Duyệt qua từng dòng của cột đó để kiểm tra cột có giá trị khác 0 hay không
        for l in range(j, len(A)):
            if A[l][i] != 0:
                flag = True
                
        if flag:
            if A[k][i] == 0:
                for z in range(k + 1, len(A)):
                    if A[z][i] != 0:
                        A[k], A[z] = A[z], A[k]
                        B[k], B[z] = B[z], B[k]
                        break
            
            d = (1 / A[k][i])
            for m in range(len(A[0])):
                A[k][m] *= d
                B[k][m] *= d
            
            for k1 in range(len(A)):
                if k1 == i:
                    continue
                d = -A[k1][i] / A[k][i]
                for m in range(len(A[0])):
                    A[k1][m] += d * A[k][m]
                    B[k1][m] += d * B[k][m]
            j += 1
        else:
            print("Ma trận không khả nghịch")
            return None
    return B
    

def newMatrix(a,i):
    arr = copy.deepcopy(a)
    if len(arr) == 2:
        return arr
    else:
        arr.pop(0) # Xóa dòng 0
        for j in arr:  # Xóa cột i
            j.pop(i)
        return arr


def multiMatrix(A,B):
    if(len(A[0])!=len(B)):
        print("Không thể nhân 2 ma trận\n")
        return None
    C = []
    ans = 0
    for i in range(len(A)):
        tam = []
        for j in range(len(A[0])):
            ans = 0
            for k in range(len(A[0])):
                ans += A[i][k] * B[k][j]
            tam.append(round(ans,3))
        C.append(tam)
    return C


def det(A):
    if len(A) != len(A[0]):
        return 0
    if len(A) == 1:
        pro = A[0]
        return pro
    elif len(A) == 2:
        pro = A[0][0]*A[1][1] - A[1][0]*A[0][1]
        return pro
        
    else:
        pro = 0
        for i in range(len(A[0])): # Tính định thức theo dòng 0
            pro += ((-1)**i)*A[0][i]*det(newMatrix(A,i))    
        return pro   



def diagonalMatrix(A):
    
    Atemp=copy.deepcopy(A)
    
    if len(A) != len(A[0]):
        print("Không chéo hóa được do ma trận input không vuông!\n");
        return None,None,None
    x = sp.symbols('x')
    for i in range(len(A)):
        A[i][i]=A[i][i]-x
    det1 = det(A)
    eq1 = sp.Eq(det1,0)
    sol = sp.solve(eq1)
    
    temp=[]
    
    for i in range(len(sol)):
        if(sp.im(sol[i])!=0): # Nếu là nghiệm ảo thì bỏ qua
            continue
        A1=copy.deepcopy(Atemp)        
        for j in range(len(A1)):
            A1[j][j]=A1[j][j]-sol[i]
        A1=sp.Matrix(A1)
        tempA1=A1.nullspace()
        
        tempans=[]
        for k in range(len(tempA1)):
            tempNS=tempA1[k].tolist()
            tempans=[]
            for l in range(len(tempNS)):
                tempans.append(tempNS[l][0])
            temp.append(copy.deepcopy(tempans))        
        
    if len(temp) != len(A):
        print("Không thể chéo hóa ma trận\n")
        return None,None,None
    P=[]
    for i in range(len(temp[0])):
        temp1=[]
        for j in range(len(temp)):
            temp1.append(temp[j][i])
        P.append(temp1)
        
    Ptemp=copy.deepcopy(P)
    P1=inverse(Ptemp)
    
    Ptemp2=multiMatrix(P1,Atemp)
    D=multiMatrix(Ptemp2,P)
    
    return P, D, P1



def printRes(name, A):
    
    print(name + ": ")
    printMatrix(A)
    
    P,D,P1 = diagonalMatrix(A)
    if P == None:
        return
    print("P:")
    printMatrix(P)
    print("D:")
    printMatrix(D)
    print("P^(-1):")
    printMatrix(P1)
    
    Ptemp=multiMatrix(P,D)
    B=multiMatrix(Ptemp,P1)
    
    print("P*D*P^(-1) = " + name + ":")
    printMatrix(B)
    return

A=[[1,0, 0],[1,2,0],[-3,5,2]]# Không thể chéo hóa được
B=[[4,0,1],[-2,1,0],[-2,0,1]]
C=[[1,3,3],[-3,-5,-3],[3,3,1]]
D=[[3,3,2],[1,1,-2],[-3,-1,0]]# Không thể chéo hóa được
print("\n---------Test 1---------")
printRes('A',A)

print("\n---------Test 2---------")
printRes('B',B)

print("\n---------Test 3---------")

printRes('C',C)

print("\n---------Test 4---------")
printRes('D',D)

   


---------Test 1---------
A: 
['1', '0', '0']
['1', '2', '0']
['-3', '5', '2']


Không thể chéo hóa ma trận


---------Test 2---------
B: 
['4', '0', '1']
['-2', '1', '0']
['-2', '0', '1']


P:
['0', '-1/2', '-1']
['1', '1', '1']
['0', '1', '1']


D:
['1', '0', '0']
['0', '2', '0']
['0', '0', '3']


P^(-1):
['0', '1', '-1']
['2', '0', '2']
['-2', '0', '-1']


P*D*P^(-1) = B:
['4', '0', '1']
['-2', '1', '0']
['-2', '0', '1']



---------Test 3---------
C: 
['1', '3', '3']
['-3', '-5', '-3']
['3', '3', '1']


P:
['-1', '-1', '1']
['1', '0', '-1']
['0', '1', '1']


D:
['-2', '0', '0']
['0', '-2', '0']
['0', '0', '1']


P^(-1):
['1', '2', '1']
['-1', '-1', '0']
['1', '1', '1']


P*D*P^(-1) = C:
['1', '3', '3']
['-3', '-5', '-3']
['3', '3', '1']



---------Test 4---------
D: 
['3', '3', '2']
['1', '1', '-2']
['-3', '-1', '0']


Không thể chéo hóa ma trận



### Bài 2)
### Mở rộng: Tìm hiểu các hàm/ phương thức tương ứng của các thư viện và thực hiện nó, so sánh kết quả.

In [35]:
import sympy as sp

def print_ans_lib(A):
    print("A: ")
    display(A)
    
    try:
        P, D = A.diagonalize()
        print("P: ")
        display(P)
        print("D: ")
        display(D)
        P_inv = P.inv()
        print("P^(-1): ")
        display(P_inv)
        print("P*D*P^(-1) = A: ")
        display(P * D * P_inv)
    except Exception as e:
        print(f"Ma trận không thể chéo hóa được")

print("\n---------Test 1---------")
A1 = sp.Matrix([[1, 0, 0], [1, 2, 0], [-3, 5, 2]])# Không thể chéo hóa được
print_ans_lib(A1)

print("\n---------Test 2---------")
A2 = sp.Matrix([[4,0,1],[-2,1,0],[-2,0,1]])
print_ans_lib(A2)

print("\n---------Test 3---------")
A3 = sp.Matrix([[1,3,3],[-3,-5,-3],[3,3,1]])
print_ans_lib(A3)

print("\n---------Test 4---------")
A4 = sp.Matrix([[3,3,2],[1,1,-2],[-3,-1,0]])# Không thể chéo hóa được
print_ans_lib(A4)



---------Test 1---------
A: 


Matrix([
[ 1, 0, 0],
[ 1, 2, 0],
[-3, 5, 2]])

Ma trận không thể chéo hóa được

---------Test 2---------
A: 


Matrix([
[ 4, 0, 1],
[-2, 1, 0],
[-2, 0, 1]])

P: 


...

Matrix([
[4,    0,   0],
[0, -2*I,   0],
[0,    0, 2*I]])

P^(-1): 


Matrix([
[     -1/2,      -1/2,   0],
[1/4 - I/4, 1/4 + I/4, 1/2],
[1/4 + I/4, 1/4 - I/4, 1/2]])

P*D*P^(-1) = A: 


Matrix([
[                                     3,                                      3,  2],
[                                     1,                                      1, -2],
[-2 - 2*I*(1/4 - I/4) + 2*I*(1/4 + I/4), -2 - 2*I*(1/4 + I/4) + 2*I*(1/4 - I/4),  0]])

### Bài 2
### Mở rộng: Tìm hiểu các hàm/ phương thức tương ứng của các thư viện và thực hiện nó, so sánh kết quả.

---

### Nhận xét:
**Đối với hàm diagonalMatrix tự định nghĩa:**
- Đối với trường hợp ma trận $A$ có thể chéo hóa được, hàm diagonalMatrix trả về ma trận làm chéo $P$, ma trận đường chéo $D$ và ma trận $P^{-1}$. Khi thực hiện $PDP^{-1}$ ta thấy kết quả rõ ràng $PDP^{-1} = A$.
- Đối với trường hợp ma trận $A$ không thể chéo hóa được, hàm diagonalMatrix thông báo lỗi, trả về ba giá trị $None$ thay cho ma trận làm chéo $P$, ma trận đường chéo $D$ và ma trận $P^{-1}$ rồi kết thúc hàm.

**Đối với hàm diagonalize của thư viện sympy:**
- Đối với trường hợp ma trận $A$ có thể chéo hóa được, hàm diagonalize của thư viện sympy trả về ma trận làm chéo $P$ và ma trận đường chéo $D$, ta tìm $P^{-1}$ bằng hàm inv của thư viện sympy. Khi thực hiện $PDP^{-1}$ ta thấy kết quả rõ ràng $PDP^{-1} = A$.
- Đối với trường hợp ma trận $A$ không thể chéo hóa được, hàm diagonalize của thư viện sympy sẽ ném ra một ngoại lệ (thường là NonSquareMatrixError hoặc MatrixError)
* Trường hợp này, SymPy sẽ thông báo lỗi rằng ma trận không thể chéo hóa, và sẽ không trả về ma trận $P$ và $D$.

**Nhận xét, so sánh kết quả:**
- Đối với trường hợp ma trận $A$ (A là đại diện cho các ma trận khác) có thể chéo hóa được, hàm diagonalize của thư viện sympy chỉ trả về ma trận làm chéo $P$ và ma trận đường chéo $D$, còn hàm diagonalMatrix tự định nghĩa trả về ma trận làm chéo $P$, ma trận đường chéo $D$ và cả ma trận $P^{-1}$. Tuy nhiên, khi thực hiện $PDP^{-1}$ thì hai hàm đều cho ra kết quả giống nhau và $PDP^{-1} = A$.

- Đối với trường hợp ma trận $A$ không thể chéo hóa được, hàm diagonalize của thư viện sympy sẽ ném ra một ngoại lệ. Tuy nhiên, ở test case 1 và test case 4 dù cả 2 là ma trận không chéo hóa được nhưng test case 4 không bị thông báo lỗi và vẫn xuất ra kết quả bình thường. Từ đó suy ra một số trường hợp, hàm diagonalize của SymPy vẫn trả về ma trận $P$ và $D$ do số lượng vector riêng độc lập không đủ để chéo hóa ma trận. Kết quả tính $PDP^{-1} có thể không bằng $A$.

----

### Mở rộng: Tìm hiểu và trình bày ứng dụng của chéo hóa
Một số ứng dụng của chéo hóa ma trận như sau:

- **1) Lý thuyết điều khiển:** Các ma trận chéo hóa được sử dụng trong lý thuyết điều khiển để thiết kế hệ thống điều khiển phản hồi. Hình dạng đường chéo của một ma trận làm cho nó dễ dàng phân tích tính ổn định và hiệu suất của hệ thống.

- **2) Tìm trị riêng và vector riêng:** trị riêng và vector riêng của một ma trận có thể được sử dụng để xác định xem một ma trận có thể chéo hóa hay không. Nếu một ma trận có n vector riêng độc lập tuyến tính, thì nó có thể chéo hóa.

- **3) Biến đổi cơ sở:** Các ma trận chéo hóa có thể được sử dụng để biến đổi một cơ sở của không gian vector thành một cơ sở mới. Việc này hữu ích trong các ứng dụng khác nhau, bao gồm xử lý tín hiệu và nén ảnh.

- **4) Phương trình vi phân:** Các ma trận chéo hóa được sử dụng để giải các hệ phương trình vi phân tuyến tính. Giải pháp cho một hệ phương trình vi phân tuyến tính có thể được biểu diễn dưới dạng các vector riêng và giá trị riêng của ma trận tương ứng.

- **5) Tính lũy thừa của ma trận:** bằng việc chéo hóa một ma trận, việc tính lũy thừa một ma trận trở nên dễ dàng hơn, đặc biệt đối với yêu cầu lũy thừa lớn. Sau khi tìm được ma trận $P$ làm chéo $A$ và $D = P^{−1}AP$ là ma trận đường chéo, việc tính $A^n$ chỉ đơn giản là tính $PD^{n}P^{−1}$, mà $D$ là ma trận đường chéo nên chỉ cận thực hiện lũy thừa $n$ các phần tử trên đường chéo của ma trận $D$

- **6) Phân tích dữ liệu:** Trong phân tích dữ liệu, các ma trận đường chéo hóa được sử dụng trong phân tích thành phần chính (PCA), đó là một kỹ thuật được sử dụng để giảm chiều của một tập dữ liệu. PCA bao gồm tìm các vector riêng và giá trị riêng của ma trận hiệp phương sai của tập dữ liệu.

- **7) Cơ học lượng tử:** Trong cơ học lượng tử, các ma trận chéo hóa được sử dụng để biểu diễn toán tử Hamiltonian, mô tả năng lượng của một hệ thống lượng tử. Các giá trị riêng và vector riêng của toán tử Hamiltonian được sử dụng để xác định các cấp năng lượng và hàm sóng của hệ thống.

Tóm lại, ma trận chéo hóa có nhiều ứng dụng quan trọng trong các lĩnh vực khác nhau như toán học, vật lý, kỹ thuật và khoa học máy tính,...

-----

####  Mô tả ý tưởng thực hiện và mô tả các hàm: 
**1) Các thư viện hỗ trợ:** trong đồ án này có dùng một số hàm của các thư viện copy và sympy để hỗ trợ các chức năng không chủ yếu trong quá trình thực hiện đồ án.
- Thư viện sympy:
    + Hàm Eq: đưa đa thức đặc trưng về phương trình đặc trưng để dùng thư viện sympy để giải phương trình này
    + Hàm symbols: dùng để đưa biến x vào phương trình đặc trưng để giải
    + Hàm tolist(): chuyển Matrix sang list
    + Hàm solve: giải phương trình đặc trưng và trả về các nghiệm
    + Hàm Matrix: tạo ma trận để dùng các thư viện hỗ trợ tìm vector riêng
    + Hàm nullspace: hỗ trợ tìm vector riêng
    + Hàm im: phần ảo của các nghiệm sau khi giải phương trình đặc trưng, nếu phần ảo khác 0 thì đó là nghiệm phức
- Thư viện copy:
    + Hàm deepcopy: tạo ra bản sao các ma trận, các vector,..
    

**2) Hàm float_to_fraction(x):** Chuyển đổi một số thực thành phân số. Nếu số thực là một số nguyên, hàm sẽ trả về số nguyên đó. Đôi khi các số như sqrt(3),... ở dạng số thực thì hàm sẽ tìm phân số gần nhất với số thực đó.

**3) Hàm printMatrix(A):** Duyệt qua từng hàng của ma trận A và in ra màn hình. Nếu A là None, in dòng trống.

**4) Hàm createIdentityMatrix(n):** tham số truyền vào là số nguyên n, hàm sẽ tạo ra ma trận đơn vị cấp n

**5) Hàm inverse(A):** truyền vào là một ma trận, hàm dùng để tìm ma trận nghịch đảo của một ma trận vuông cấp n
- B1: Đầu tiên, kiểm tra ma trận truyền vào có vuông hay không, nếu không vuông thì kết thúc hàm
- B2: Nếu ma trận input là ma trận vuông cấp n, tiến hành khởi tạo một ma trận đơn vị B cấp n bằng hàm createIdentityMatrix
- B3: Duyệt qua từng cột của ma trận input, nếu cột đang xét có phần tử nằm từ đường chéo chính trở xuống đều bằng 0 thì kết thúc hàm, thông báo ma trận không khả nghịch, ngược lại thì thực hiện các bước sau:
    + Kiểm tra nếu phần tử nằm trên đường chéo chính bằng không thì hoán vị dòng chứa phần tử này với dòng có phần tử ở cột tương ứng khác 0 gần nhất với dòng đang xét. Việc này được thực hiện cùng lúc ở ma trận input và ma trận đơn vị B đã tạo ra trước đó.
    + Nhân dòng đang xét của ma trận input và dòng tương ứng của ma trận đơn vị B với 1/(phần tử ở đường chéo chính của ma trận input).
    + Cộng các dòng còn lại của ma trận input và cả ma trận đơn vị B (trừ dòng chứa phần tử nằm trên đường chéo chính) với một số thích hợp để biến các phần tử ở cột đang xét của ma trận input đều bằng 0.
    + Lặp lại B3
- B4: Ma trận đơn vị B được tạo ra ở B2 sau khi thực hiện cả 3 bước trên sẽ là ma trận nghịch đảo của ma trận input, hàm sẽ trả về ma trận B

**6) Hàm multiMatrix(A,B):** hàm nhân hai ma trận, với input là ma trận A và B, Hàm sẽ kiểm tra xem số cột của A có bằng số hàng của B hay không. Nếu có, thực hiện phép nhân ma trận. Nếu không, in thông báo lỗi và trả về None.

**7) Hàm newMatrix(a,i):** input là ma trận vuông A và số nguyên i. Nếu A là ma trận vuông cấp 2 thì trả về ma trận A, ngược lại hàm tạo ra một ma trận mới bỏ đi dòng 0 và cột i và trả về ma trận này (hàm hỗ trợ tìm định thức của ma trận trong hàm det(A)).

**8) Hàm det(A):** hàm tính định thức của ma trận
- Input: ma trận vuông A cấp n
- B1: Kiểm tra ma trận A có là ma trận vuông hay không, nếu không thì trả về số nguyên 0 và kết thúc hàm, ngược lại thì tiếp tục sang B2
- B2: Nếu A là ma trận vuông cấp 1, trả về A[0][0] (là định thức của ma trận) và kết thúc hàm, ngược lại thì tiếp tục sang B3
- B3: Nếu A là ma trận vuông cấp 2, trả về A[0][0] * A[1][1] - A[1][0] * A[0][1] là định thức của ma trận và kết thúc hàm, ngược lại thì tiếp tục sang B4
- B4: Dùng công thức $\sum_{k=0}^{n-1} a_{0k}C_{0k} (1)$ để tính định thức theo dòng 0, cộng dồn vào biến pro. Ý tưởng thực hiện: cho một vòng for i duyệt qua các cột của ma trận, gọi đệ quy hàm det(X) với X là ma trận input mỗi lần gọi là ma trận hiện tại bỏ dòng 0 cột i hoặc chính ma trận hiện tại nếu đang là ma trận vuông cấp 2 (dùng hàm newMatrix) nhân với $-1^{0+i} * A[0][i]$ theo đúng công thức $(1)$ và trả về pro là định thức của ma trận

**9) Hàm diagonalMatrix:** input là ma trận vuông A cấp n, hàm tìm và trả về ma trận làm chéo $P$, ma trận đường chéo $D$ và ma trận $P^{-1}$.
- B1: Đầu tiên dùng hàm copy.deepcopy tạo một bản sao của A và lưu vào Atemp (có thêm tài nguyên để tính toán), kiểm tra ma trận có vuông hay không, nếu ma trận không vuông thì trả về 3 giá trị None và kết thúc hàm, ngược lại sang B2
- B2: Khởi tạo biến x bằng hàm sympy.symbol để đưa vào đa thức đặc trưng (biến x đại diện cho $λ$)
- B3: Dùng vòng for xây dựng ma trận $A − xI_n$ và lưu vào A
- B4: Tính định thức ma trận A bằng hàm det(A) và lưu vào det1
- B5: Chuyển det1 thành phương trình đặc trưng bằng hàm sympy.Eq và lưu vào eq1
- B6: Giải phương trình eq1 bằng hàm sympy.solve và lưu các nghiệm vào sol
- B7: Chạy vòng for xét từng nghiệm trong sol, với mỗi nghiệm dùng hàm sympy.im để tìm phần ảo của nghiệm, nếu phần ảo khác 0 thì là nghiệm phức, ta không làm gì cả và tiếp tục tăng biến đếm vòng for (continue), ngược lại thực hiện các bước sau:
    + B7.1: Tạo ra một bản copy của Atemp bằng hàm copy.deepcopy (do A ban đầu đã bị thay đổi do đã đưa biến x vào ma trận để tính định thức) và lưu vào A1
    + B7.2: Với nghiệm $x_i$ đang xét dùng vòng for để xây dựng ma trận $A1 − x_iI_n$ và lưu vào A1. Sau đó chuyển A sang dạng sympy.Matrix
    + B7.3: Với $x_i$, tìm các vector riêng của $(A − x_iI_n) X = 0$ bằng hàm sympy.Matrix.nullspace, sau đó chuyển các vector riêng này từ kiểu sympy.Matrix sang list bằng sympy.Matrix.tolist (tiện hơn cho việc tính toán về sau)
- B8: Kiểm tra nếu số lượng vector riêng khác cấp n của ma trận input thì trả về 3 giá trị None và kết thúc hàm, ngược lại sang B9
- B9: Tạo ma trận làm chéo $P$ là ma trận có được bằng cách dựng các vector riêng thành các cột và đưa vào $P$.
- B10: Sao chép $P$ và lưu vào Ptemp bằng hàm copy.deepcopy. Tìm $P^{-1}$ bằng hàm inverse và lưu vào $P1$
- B11: Tìm ma trận đường chéo D bằng cách nhân $P1 * Atemp * P$ Đầu tiên thực hiện nhân $P1 * Atemp$ bằng hàm multiMatrix và lưu vào $Ptemp2$, sau đó thực hiện nhân $Ptemp2 * P$ bằng hàm multiMatrix và lưu vào D
- B12: Trả về $P, D, P1$ tương ứng với ma trận làm chéo $P$, ma trận đường chéo $D$ và ma trận $P^{-1}$

**10) Hàm printRes:** hàm in kết quả test chương trình chính (chương trình dùng hàm diagonalMatrix), input là ma trận A
- B1: In ra ma trận A bằng hàm printMatrix
- B2: Tìm ma trận làm chéo $P$, ma trận đường chéo $D$ và ma trận $P^{-1}$ bằng hàm diagonalMatrix
- B3: Nếu sau khi thực hiện hàm diagonalMatrix mà $P = None$ nghĩa là ma trận A không chéo hóa được và kết thúc hàm, ngược lại chuyển sang B4
- B4: In ra ma trận $P, D, P^{-1}$ sau khi thực hiện hàm diagonalMatrix (in bằng hàm printMatrix)
- B5: Thực hiện tính và in ra $PDP^{-1}$ bằng các hàm multiMatrix (tính $P * D$ trước và lưu vào $Ptemp$, sau đó tính $Ptemp * P^{-1}$) và printMatrix để kiểm tra $PDP^{-1} = A$.

**11) Hàm print_ans_lib:** hàm in kết quả test chương trình mở rộng (chương trình dùng hàm sympy.Matrix.diagonalize), input là ma trận A kiểu sympy.Matrix
- B1: In ra ma trận A bằng hàm display
- B2: Tìm ma trận làm chéo $P$ và ma trận đường chéo $D$ bằng hàm sympy.Matrix.diagonalize, in ra màn hình $P$ và $D$ bằng hàm display
- B3: Tìm $P^{-1}$ bằng hàm sympy.Matrix.inv và in ra màn hình $P^{-1}$ bằng hàm display
- B4: Tính và in ra màn hình $PDP^{-1}$ bằng toán tử $*$ và hàm display để kiểm tra $PDP^{-1} = A$.
* Mục đích chính của try-catch: Để bắt, xử lý các ngoại lệ (exceptions) có thể xảy ra trong quá trình thực hiện chéo hóa ma trận. Điều này giúp ngăn chương trình bị dừng đột ngột và cung cấp thông tin cụ thể về lỗi cho người dùng.

----